In [ ]:


df_clean_cut_Ideal = df_clean.query("cut =='Ideal'")
df_clean_cut_Premium = df_clean.query("cut =='Premium'")
df_clean_cut_Good = df_clean.query("cut =='Good'")
df_clean_cut_Very_Good = df_clean.query("cut =='Very Good'")
df_clean_cut_Fair = df_clean.query("cut =='Fair'")


cut_df_list = [df_clean_cut_Ideal,df_clean_cut_Premium,df_clean_cut_Good,df_clean_cut_Very_Good,df_clean_cut_Fair]

In [ ]:
for i in cut_df_list:

  # 1) 그래프 초기화 (캔버스(fig)와 도화지(ax) 준비하기)
  width_px = 1280              # 그래프 가로 크기
  height_px = 720              # 그래프 세로 크기
  rows = 2                     # 그래프 행 수
  cols = 3                     # 그래프 열 수
  figsize = (width_px / my_dpi, height_px / my_dpi)
  fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

  # 2) 그래프 그리기 -> seaborn 사용
  sb.boxplot(data=i, y='price',ax=ax[0][0])
  sb.boxplot(data=i, y='depth',ax=ax[0][1])
  sb.boxplot(data=i, y='table',ax=ax[0][2])


  sb.boxplot(data=i, y='x',ax=ax[1][0])
  sb.boxplot(data=i, y='y',ax=ax[1][1])
  sb.boxplot(data=i, y='z',ax=ax[1][2])


  # 3) 그래프 꾸미기

  #그래프 전체에 대한 제목
  fig.suptitle(f'{i['cut'].iloc[0]} 다이아몬드 데이터 박스 플롯', fontsize=14, color='#006600')


  #각 그래프 간의 가로 (wspace) , 세로 (hspace) 간격 지정
  fig.subplots_adjust(wspace=0.2 , hspace=0.2)

  # 그래프의 제목, 글자크기, 색상, 격자
  ax[0][0].set_title("price", color="#0066ff", fontsize=8, pad=8)
  ax[0][0].grid()

  ax[0][1].set_title("depth", color="#0066ff", fontsize=8, pad=8)
  ax[0][1].grid()


  ax[0][2].set_title("table", color="#0066ff", fontsize=8, pad=8)
  ax[0][2].grid()

  ax[1][0].set_title("x", color="#0066ff", fontsize=8, pad=8)
  ax[1][0].grid()

  ax[1][1].set_title("y", color="#0066ff", fontsize=8, pad=8)
  ax[1][1].grid()

  ax[1][2].set_title("z", color="#0066ff", fontsize=8, pad=8)
  ax[1][2].grid()



  # 4) 출력
  plt.tight_layout()                     # 여백 제거
  plt.show()                             # 그래프 화면 출력
  plt.close()                            # 그래프 작업 종료


============================

In [ ]:
df_clean['volume'] = df_clean['x'] * df_clean['y'] * df_clean['z']
df_clean


In [ ]:
data = df_wide[['Ideal','Premium']]                # 확인할 데이터

# 1) 그래프 초기화
width_px  = 1280                      # 그래프 가로 크기
height_px = 720                       # 그래프 세로 크기
rows = 1                              # 그래프 행 수
cols = 1                              # 그래프 열 수
figsize = (width_px / my_dpi, height_px / my_dpi)
fig, ax = plt.subplots(rows, cols, figsize=figsize, dpi=my_dpi)

for c in data.columns:
    # KDE Plot 그리기
    sb.kdeplot(data=data, x=c, fill=False, alpha=0.5)

    # 신뢰구간
    max = data[c].max()               # 최대값
    clevel = 0.95                     # 신뢰수준
    dof = len(data[c]) - 1            # 자유도
    sp_mean = data[c].mean()          # 표본평균
    sp_std = data[c].std(ddof=1)      # 표본표준편차
    sp_std_error = sp_std / sqrt(len(data[c]))  # 표본표준오차

    cmin, cmax = t.interval(clevel, dof, loc=sp_mean, scale=sp_std_error)

    ax.axvline(cmin, linestyle=':', linewidth=0.5)
    ax.axvline(cmax, linestyle=':', linewidth=0.5)

    # 평균 표시
    ymin, ymax = ax.get_ylim()
    ax.fill_between([cmin, cmax], 0, ymax, alpha=0.1)
    ax.axvline(sp_mean, linestyle='--', linewidth=1)

    # 텍스트 그리기
    ax.text(x=(cmax-cmin)/2+cmin,
            y=ymax,
            s=f"[{c}] {cmin:.1f} ~ {cmax:.1f} ({sp_mean:.1f})",
            horizontalalignment="center",
            verticalalignment="bottom",
            fontdict={"size": 6, "color": "red"})

# 3) 그래프 꾸미기
ax.grid(True, alpha=0.3)

# 4) 출력
plt.tight_layout()      # 여백 제거
plt.show()              # 그래프 화면 출력
plt.close()             # 그래프 작업 종료


------------------

In [ ]:
xname = ['cut' , 'color']
yname = 'log_price'
data = df_log[xname + [yname]]

if equal_var:
    pg_anova = anova(data=df_log, dv=yname, between=xname)
else:
    pg_anova = welch_anova(data=df_log, dv=yname, between=xname)
display(pg_anova)
for idx in pg_anova.index:
    row = pg_anova.iloc[idx]
    source = row['Source']
    p_val = row['p-unc']

    if p_val <= 0.05:
      print(f"→ {source} 효과는 유의함. (대립가설 채택: p ≤ 0.05)")

    else:
      print(f"→ {source} 효과는 유의하지 않음. (귀무가설 채택: p > 0.05)")